In [1]:
import warnings
warnings.filterwarnings('ignore')

from keras.layers import Input, Dense, Flatten, Dropout
from keras.models import Model

import numpy as np
import pandas as pd
import os

import random
from collections import deque
import matplotlib.pylab as plt

from sklearn.decomposition import PCA

In [2]:
# TODO: add other metrics: returns, volatility, alpha, beta to the benchmark

# Imports of classes

In [3]:
from environment import ETFEnvironment, CryptoEnvironment
from agent import MinVarianceAgent, MaxSharpeAgent, MaxDecorrelationAgent, MaxReturnsAgent
from utils import *

In [4]:
N_ASSETS = 53
WINDOW_FIT = 180
WINDOW_HOLD = 180
env = ETFEnvironment()

In [5]:
agent_max_returns = MaxReturnsAgent(N_ASSETS, allow_short=True)
agent_minvar = MinVarianceAgent(N_ASSETS, allow_short=True)
agent_maxsharpe = MaxSharpeAgent(N_ASSETS, allow_short=True)
agent_maxdecorr = MaxDecorrelationAgent(N_ASSETS, allow_short=True)

# Reweighting process: optimization

In [ ]:
actions_equal, actions_returns, actions_minvar, actions_maxsharpe, actions_maxdecorr = [], [], [], [], []
result_equal, result_returns, result_minvar, result_maxsharpe, result_maxdecorr = [], [], [], [], []

for i in range(WINDOW_FIT, len(env.data), WINDOW_HOLD):
    
    state = env.get_state(i, WINDOW_FIT, is_cov_matrix=False)
    
    action_equal = np.ones(N_ASSETS) / N_ASSETS
    action_minvar = agent_minvar.act(state)
    action_max_returns = agent_max_returns.act(state)
    action_maxsharpe = agent_maxsharpe.act(state)
    action_maxdecorr = agent_maxdecorr.act(state)
 
    state_action = env.get_state(i+WINDOW_HOLD, WINDOW_HOLD, is_cov_matrix=False)
    
    r = np.dot(state_action, action_equal)
    result_equal.append(r.tolist())
    actions_equal.append(action_equal)
    
    r = np.dot(state_action, action_minvar)
    result_minvar.append(r.tolist())
    actions_minvar.append(action_minvar)

    r = np.dot(state_action, action_max_returns)
    result_returns.append(r.tolist())
    actions_returns.append(action_max_returns)
    
    r = np.dot(state_action, action_maxsharpe)
    result_maxsharpe.append(r.tolist())
    actions_maxsharpe.append(action_maxsharpe)
    
    r = np.dot(state_action, action_maxdecorr)
    result_maxdecorr.append(r.tolist())
    actions_maxdecorr.append(action_maxdecorr)

# Evaluation

In [ ]:
result_equal_vis = [item for sublist in result_equal for item in sublist]
result_returns_vis = [item for sublist in result_returns for item in sublist]
result_minvar_vis = [item for sublist in result_minvar for item in sublist]
result_maxsharpe_vis = [item for sublist in result_maxsharpe for item in sublist]
result_maxdecorr_vis = [item for sublist in result_maxdecorr for item in sublist]

In [ ]:
%matplotlib inline

plt.figure()
plt.plot(np.array(result_equal_vis).cumsum(), label = 'result_equal_vis')
plt.plot(np.array(result_minvar_vis).cumsum(), label = "result_minvar_vis")
plt.plot(np.array(result_returns_vis).cumsum(), label = "result_returns_vis")
plt.plot(np.array(result_maxsharpe_vis).cumsum(), label = "result_maxsharpe_vis")
plt.plot(np.array(result_maxdecorr_vis).cumsum(), label = "result_maxdecorr_vis")
plt.legend(["result_equal_vis", "result_minvar_vis", "result_returns_vis", "result_maxsharpe_vis", "result_maxdecorr_vis"])
plt.show()




In [ ]:
print('EQUAL', print_stats(result_equal_vis, result_equal_vis))
print('MINVAR', print_stats(result_minvar_vis, result_equal_vis))
print('MAXRET', print_stats(result_returns_vis, result_equal_vis))
print('MAXSHRAPE', print_stats(result_maxsharpe_vis, result_equal_vis))
print('MAXDECORR', print_stats(result_maxdecorr_vis, result_equal_vis))

# Visualization

In [ ]:
import matplotlib
current_cmap = matplotlib.cm.get_cmap()
current_cmap.set_bad(color='red')

In [ ]:
def plot_results(benchmark_series, 
                 target_series, 
                 target_balances, 
                 n_assets = N_ASSETS,
                 columns = state.columns,
                 name2plot = '',
                 path2save = './',
                 base_name_series = 'series'):
    
#     N = len(np.array(benchmark_series).cumsum())
    N = len(np.array([item for sublist in benchmark_series for item in sublist]).cumsum()) 
    
    if not os.path.exists(path2save):
        os.makedirs(path2save)

    for i in range(0, len(target_balances)):

        current_range = np.arange(0, N)
        current_ts = np.zeros(N)
        current_ts2 = np.zeros(N)

        ts_benchmark = np.array([item for sublist in benchmark_series[:i+1] for item in sublist]).cumsum()
        ts_target = np.array([item for sublist in target_series[:i+1] for item in sublist]).cumsum()

        t = len(ts_benchmark)
        current_ts[:t] = ts_benchmark
        current_ts2[:t] = ts_target

        current_ts[current_ts == 0] = ts_benchmark[-1]
        current_ts2[current_ts2 == 0] = ts_target[-1]

        plt.figure(figsize = (12, 10))
        
        plt.subplot(2, 1, 1)
        plt.bar(np.arange(n_assets), target_balances[i], color = 'grey')
        plt.xticks(np.arange(n_assets), columns, rotation='vertical')

        plt.subplot(2, 1, 2)
        plt.colormaps = current_cmap
        plt.plot(current_range[:t], current_ts[:t], color = 'black', label = 'Benchmark')
        plt.plot(current_range[:t], current_ts2[:t], color = 'red', label = name2plot)
        plt.plot(current_range[t:], current_ts[t:], ls = '--', lw = .1, color = 'black')
        plt.autoscale(False)
        plt.ylim([-1.5, 1.5])
        plt.legend()
        plt.savefig(path2save + base_name_series + str(i) + '.jpg')

In [ ]:
plot_results(result_equal, 
             result_maxdecorr, 
             actions_maxdecorr, 
             N_ASSETS,
             state.columns.tolist(), 
             'Decorrelation portfolio', './images_etf/decorr/', 'series')

In [ ]:
plot_results(result_equal, 
             result_maxsharpe, 
             actions_maxsharpe,  
             N_ASSETS,
             state.columns.tolist(), 
             'Maximal Sharpe portfolio', './images_etf/sharpe/', 'series')

In [ ]:
plot_results(result_equal, 
             result_minvar, 
             actions_minvar,  
             N_ASSETS,
             state.columns.tolist(), 
             'Minimal variance portfolio', './images_etf/minvar/', 'series')

In [ ]:
plot_results(result_equal, 
             result_returns, 
             actions_returns,  
             N_ASSETS,
             state.columns.tolist(), 
             'Maximal returns portfolio', './images_etf/maxret/', 'series')

# Reweighting process: PCA

In [ ]:
from agent import PCAAgent

In [ ]:
agent_pca = PCAAgent(N_ASSETS, allow_short=True, pc_id=0)

In [ ]:
actions_equal, actions_pca = [], []
result_equal, result_pca = [], []

for i in range(WINDOW_FIT, len(env.data), WINDOW_HOLD):
    
    state = env.get_state(i, WINDOW_FIT, is_cov_matrix=False)
    
    action_equal = np.ones(N_ASSETS) / N_ASSETS
    action_pca = agent_pca.act(state)
 
    state_action = env.get_state(i+WINDOW_HOLD, WINDOW_HOLD, is_cov_matrix=False)
    
    r = np.dot(state_action, action_equal)
    result_equal.append(r.tolist())
    actions_equal.append(action_equal)
    
    r = np.dot(state_action, action_pca)
    result_pca.append(r.tolist())
    actions_pca.append(action_pca)

# Evaluation

In [ ]:
result_equal_vis = [item for sublist in result_equal for item in sublist]
result_pca_vis = [item for sublist in result_pca for item in sublist]

In [ ]:
plt.figure()
plt.plot(np.array(result_equal_vis).cumsum())
plt.plot(np.array(result_pca_vis).cumsum())
plt.show()

In [ ]:
print('EQUAL', print_stats(result_equal_vis, result_equal_vis))
print('PCA', print_stats(result_pca_vis, result_equal_vis))

# Visualization

In [ ]:
import matplotlib
current_cmap = matplotlib.cm.get_cmap()

In [ ]:
plot_results(result_equal, 
             result_pca, 
             actions_pca, 
             N_ASSETS,
             state.columns, 
             'PCA PC0 portfolio', './images_etf/pca/', 'series')

# Reweighting process: HRP

In [ ]:
from agent import HRPAgent

In [ ]:
agent_hrp = HRPAgent(N_ASSETS, allow_short=True)

In [ ]:
actions_equal, actions_hrp = [], []
result_equal, result_hrp = [], []

for i in range(WINDOW_FIT, len(env.data), WINDOW_HOLD):
    
    state = env.get_state(i, WINDOW_FIT, is_cov_matrix=False)
    
    action_equal = np.ones(N_ASSETS) / N_ASSETS
    action_hrp = agent_hrp.act(state)

    state_action = env.get_state(i+WINDOW_HOLD, WINDOW_HOLD, is_cov_matrix=False)
    
    r = np.dot(state_action, action_equal)
    result_equal.append(r.tolist())
    actions_equal.append(action_equal)
    
    r = np.dot(state_action, action_hrp)
    result_hrp.append(r.tolist())
    actions_hrp.append(action_hrp)

In [ ]:
result_equal_vis = [item for sublist in result_equal for item in sublist]
result_hrp_vis = [item for sublist in result_hrp for item in sublist]

# Evaluation

In [ ]:
plt.figure()
plt.plot(np.array(result_equal_vis).cumsum())
plt.plot(np.array(result_hrp_vis).cumsum())
plt.show()

In [ ]:
print('EQUAL', print_stats(result_equal_vis, result_equal_vis))
print('HRP', print_stats(result_hrp_vis, result_equal_vis))

# Visualization

In [ ]:
import matplotlib
current_cmap = matplotlib.cm.get_cmap()

In [ ]:
plot_results(result_equal, 
             result_hrp, 
             actions_hrp, 
             N_ASSETS,
             state.columns, 
             'HRP portfolio', './images_etf/hrp/', 'series')

# Reweighting process: Forecasting 

In [ ]:
from agent import SmoothingAgent

In [ ]:
agent_smooth = SmoothingAgent(N_ASSETS, allow_short=True, forecast_horizon = WINDOW_HOLD)

In [ ]:
actions_equal, actions_smooth = [], []
result_equal, result_smooth = [], []

for i in range(WINDOW_FIT, len(env.data), WINDOW_HOLD):
    
    state = env.get_state(i, WINDOW_FIT, is_cov_matrix=False, is_raw_time_series=True)
    
    action_equal = np.ones(N_ASSETS) / N_ASSETS
    action_smooth = agent_smooth.act(state)

    state_action = env.get_state(i+WINDOW_HOLD, WINDOW_HOLD, is_cov_matrix=False)
    
    r = np.dot(state_action, action_equal)
    result_equal.append(r.tolist())
    actions_equal.append(action_equal)
    
    r = np.dot(state_action, action_smooth)
    result_smooth.append(r.tolist())
    actions_smooth.append(action_smooth)

In [ ]:
result_equal_vis = [item for sublist in result_equal for item in sublist]
result_smooth_vis = [item for sublist in result_smooth for item in sublist]

In [ ]:
plt.figure()
plt.plot(np.array(result_equal_vis).cumsum())
plt.plot(np.array(result_smooth_vis).cumsum())
plt.show()

In [ ]:
print('EQUAL', print_stats(result_equal_vis, result_equal_vis))
print('SMOOTHING', print_stats(result_smooth_vis, result_equal_vis))

In [ ]:
plot_results(result_equal, 
             result_smooth, 
             actions_smooth,
             N_ASSETS,
             state.columns, 
             'Holt smoothing portfolio', './images_etf/smoothing/', 'series')

# Reweighting process: Autoencoder

In [ ]:
from agent import AutoencoderAgent

In [ ]:
agent_ae = AutoencoderAgent(N_ASSETS, allow_short=True, encoding_dim = 5)

In [ ]:
actions_equal, actions_ae = [], []
result_equal, result_ae = [], []

for i in range(WINDOW_FIT, len(env.data), WINDOW_HOLD):
    
    state = env.get_state(i, WINDOW_FIT, is_cov_matrix=False, is_raw_time_series=True)
    
    action_equal = np.ones(N_ASSETS) / N_ASSETS
    action_ae = agent_ae.act(state)

    state_action = env.get_state(i+WINDOW_HOLD, WINDOW_HOLD, is_cov_matrix=False)
    
    r = np.dot(state_action, action_equal)
    result_equal.append(r.tolist())
    actions_equal.append(action_equal)
    
    r = np.dot(state_action, action_ae)
    result_ae.append(r.tolist())
    actions_ae.append(action_ae)

In [ ]:
result_equal_vis = [item for sublist in result_equal for item in sublist]
result_ae_vis = [item for sublist in result_ae for item in sublist]

In [ ]:
plt.figure()
plt.plot(np.array(result_equal_vis).cumsum())
plt.plot(np.array(result_ae_vis).cumsum())
plt.show()

In [ ]:
print('EQUAL', print_stats(result_equal_vis, result_equal_vis))
print('AUTOENCODER', print_stats(result_ae_vis, result_equal_vis))

In [ ]:
plot_results(result_equal, 
             result_ae, 
             actions_ae, 
             N_ASSETS,
             state.columns, 
             'Autoencoder portfolio', './images_etf/ae/', 'series')

# Making GIFs

In [ ]:
import imageio
import glob
import os

In [ ]:
name = 'decorr'

In [ ]:
filenames = glob.glob('./images_etf/' + name + '/series*.jpg')

In [ ]:
filenames.sort(key=lambda x: os.path.getmtime(x))

In [ ]:
filenames

In [ ]:
images = []
for filename in filenames:
    images.append(imageio.imread(filename))
imageio.mimsave('./images_etf/' + name + '_gif.gif', images, duration=0.5)